<a href="https://colab.research.google.com/github/Existanze54/sirius-machine-learning-2025/blob/main/Homeworks/GenTech/HW3_LogReg_SVM_GT25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашка 3. Logistic Regression and SVM

### imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as LogReg
from sklearn.linear_model import LogisticRegressionCV as LogRegCV

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## Задача 1. Классификация на нуклеотидных k-мерах

**Protein Binding Microarray (PBM)** — один из методов изучения связывания транскрипционных факторов с ДНК. Результаты таких экспериментов позволяют устанавливать мотивы связывания, вроде этого:  
<img src="https://mex.autosome.org/motifs/motif401371.svg" width='300'>

Обычно мотивы объясняют экспериментальные результаты довольно хорошо, но порой алгоритмы ML могут найти более сложные зависимости. В данной задаче предлагается классифицировать олигонуклеотиды на связывающиеся и несвязывающиеся (согласно результатам PBM) с транскрипционным фактором [GCM1](https://www.uniprot.org/uniprotkb/Q9NP62/entry) человека.

In [ ]:
%%bash
FILEID=1tmbIBuPXhPMjcvAgfs6DFWakOjH6We7O
wget -q --no-check-certificate "https://docs.google.com/uc?export=download&id=$FILEID" -O gcm1.tsv

### Посмотрим на данные и представим их

In [ ]:
df = pd.read_csv('gcm1.tsv', sep='\t')
df.head(2)

`pbm_sequence` это тот самый изучаемый олигонуклеотид, а `mean_signal_intensity` — наша целевая переменная.  
  
До успеха нейросетей, довольно популярным представлением для текстов был **"мешок слов"** — вектор с числом всех встреченных слов. Вам предлагается представлять нуклеотидную последовательность как **"мешок k-меров"**. Для этого я написал несколько функций ниже, можете с ними ознакомиться. Мы будем учитывать все **k-меры** до длины **k = 5** (из-за ограниченности вычислительных ресурсов).

In [ ]:
# @title Функции для кодирования нуклеотидной последовательности
nucleomap = {'A': 'T',
             'C': 'G',
             'G': 'C',
             'T': 'A'}

def get_complement(seq):
    """Получает нуклеотидную последовательность
       и возвращает комплементарную ей
    """
    return ''.join(nucleomap[x] for x in seq)[::-1]

def get_canonical(kmer):
    """Возвращает лексикографически меньший k-мер
       из двух комплементарных
    """
    rc = get_complement(kmer)
    return min(kmer, rc)


def encode_kmers(seq, k=5):
    """Кодирует нуклеотидную последовательность
       как словарь каноничных k-меров
    """
    d = {}
    for L in range(1, k+1):
        for i in range(len(seq)-L+1):
            kmer = seq[i:i+L]
            can = get_canonical(kmer)
            d[can] = d.get(can, 0) + 1
    return d

In [ ]:
seqs = df['pbm_sequence']
kmer_dicts = [encode_kmers(s) for s in seqs]
X = pd.DataFrame(kmer_dicts)
X = X.fillna(0)
X.head(2)

Сигнал PBM является вещественной величиной, пропорциональной связыванию белка с олигонуклеотидом. Однако он считается довольно шумным, и связавшимися часто считают только последовательности, давшие сигнал больше 4. Такие мы будем считать положительным классом.

In [ ]:
y = df['mean_signal_intensity']
y = y > 4

### Ваше решение

Удалите k-меры, которые встречаются менее чем у 5% олигонуклеотидов. Разделите выборку на обучающую, валидационную и тестовую. Обращаю внимание, что положительного класса у нас всего 1.5%, так что лучше стратифицировать. Стандартизуйте признаки. Обучите логистическую регрессию с L2-регуляризацией. Подберите параметр `C` в диапазоне $[0.01, 100]$ по валидационной выборке. Оцените качество модели с помощью $AUROC$ и $AUPRC$. Сравните с `LogRegCV` с параметрами по умолчанию.

In [ ]:
# your code here

## Задача 2. Классификация молекул

Поработаем с известным датасетом [BACE](https://pubmed.ncbi.nlm.nih.gov/27689393/), содержащим молекулы размеченные как ингибиторы и не-ингибиторы человеческой  
*β-секретазы 1*. Дескрипторы (так хемоинформатики называют признаки) рассчитаны за вас.

In [ ]:
! wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1SYdyTbPD01GYcdz7hkyWZGoA5RV44fg7' -O bace.csv

In [ ]:
df = pd.read_csv('bace.csv')
X = df.iloc[:, 4:]
y = df['Class']

Проведите необходимую подготовку данных. Обучите `LogReg` с L1-регуляризацией, а так же `SVC` с линейным и RBF-ядром. Подберите параметр `C` для всех трех моделей. Оцените качество моделей метрикой $F_1$.

In [ ]:
# your code here